<h1 style="color:blue;text-align:center">Game of Life Utils</h1> 

<a id="tdm"></a>
<h2 style="color:green"> Table des matières </h2>

<a href="#Moteur" style="color:black;text-decoration:none;"><span style="color:#337ab7;text-decoration: underline;margin-left:0em;font-weight:bold;">Partie Moteur du Jeu</span></a>
<br>
<a href="#Interface" style="color:black;text-decoration:none;"><span style="color:#337ab7;text-decoration: underline;margin-left:0em;font-weight:bold;">Partie Interface</span></a>
<br>
<a href="#Save" style="color:black;text-decoration:none;"><span style="color:#337ab7;text-decoration: underline;margin-left:0em;font-weight:bold;">Partie Sauvegarde et Chargement</span></a>
<br>
<a href="#BoutonPause" style="color:black;text-decoration:none;"><span style="color:#337ab7;text-decoration: underline;margin-left:0em;font-weight:bold;">Bouton pause</span></a>
<br>

<a id="Moteur"></a>
<h2 style="color:Green">Fonctions Moteur du Jeu</h2>

[Retour à la Table des matières](#tdm)

In [ ]:
#####################################################################################

def grille():
    """ Calcule et dessine la nouvelle grille """
    protocole() ,draw(), draw_button_pause()
    pygame.display.flip()  # Rafraîchit l'affichage

def initialisation(dimension):
    """ Initialisation | Renvoie la hauteur, la largeur et les matrices des etats """
    hauteur, largeur = dimension,dimension
    
    # Matrice qui gèrera l'aspect graphique des cellules (rectangles)
    cellule = [[None for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient l'état actuel de chaque cellules [ 0=morte / 1=vivante], elles sont toutes mortes au début
    etat = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    # Matrice qui contient le prochain état de chaque cellule (la matrice précedente prendra sa valeur)
    etat_futur = [[0 for _ in range(hauteur)] for _ in range(largeur)]
    
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = 0
            etat_futur[x][y] = 0
            # Création des rectangles
            cellule[x][y] = pygame.Rect(x*taille_cellule, y*taille_cellule, taille_cellule, taille_cellule) 
            
    # Place environ 25% de cellules vivantes
    for i in range(largeur * hauteur // 4):
        etat[randrange(largeur)][randrange(hauteur)] = 1
        
    return hauteur, largeur, cellule, etat, etat_futur

def initialisation_aleatoire():
    """ Initialisation aléatoire de la grille : intervalle régulable """
    dimension = random.randint(50, 300)
    hauteur, largeur, cellule, etat, etat_futur = initialisation(dimension)
    return dimension, hauteur, largeur, cellule, etat, etat_futur

def protocole():
    """ Calcule et applique les règles """
    for y in range(hauteur):
        for x in range(largeur):
            nombre_voisins = compte_voisins_vivant(x, y)
            # Applique les règles de survie et de naissance des cellules
            if etat[x][y] == 1 and nombre_voisins < 2:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and nombre_voisins > 3:
                etat_futur[x][y] = 0
            elif etat[x][y] == 1 and (nombre_voisins == 2 or nombre_voisins == 3):
                etat_futur[x][y] = 1
            elif etat[x][y] == 0 and nombre_voisins == 3:
                etat_futur[x][y] = 1
        
    for y in range(hauteur):
        for x in range(largeur):
            etat[x][y] = etat_futur[x][y]

def compte_voisins_vivant(x, y):
    """ Calcul et renvoie le nombre de cellules voisines en vie (tableau torique) """
    nb_voisins = 0
    # On definit une matrice contenant les coordonnées de tous les voisins d'une cellule x=0 et y=0
    directions = [(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0, -1), (1, -1)]
    
    for dx, dy in directions:
        # on utilise le % pour que le bord gauche soit connecté au bord droit, 
        # et le bord supérieur soit connecté au bord inférieur
        if etat[(x + dx) % largeur][(y + dy) % hauteur] == 1:
            nb_voisins += 1
    return nb_voisins

#####################################################################################

<a id="Interface"></a>
<h2 style="color:Green">Partie Interface</h2>

[Retour à la Table des matières](#tdm)

In [ ]:
#####################################################################################

def draw():
    """ Dessine / Redessine toutes les cellules """
    fenetre.fill((174, 214, 241))  # Fond
    
    for y in range(hauteur):
        for x in range(largeur):
            couleur = (0, 0, 0) if etat[x][y] == 1 else (174, 214, 241)
            pygame.draw.rect(fenetre, couleur, cellule[x][y])
    
    # Dessiner le cadre noir à droite
    cadre_largeur = largeur_panel_droit  # Largeur du cadre
    cadre_rect = pygame.Rect(taille_cellule * largeur, 0, cadre_largeur, taille_cellule * hauteur)
    # Dessiner le rectangle blanc à l'intérieur du cadre
    centre_rect = pygame.Rect(taille_cellule * largeur + 10, 10, cadre_largeur - 20, taille_cellule * hauteur - 20)
    pygame.draw.rect(fenetre, (40, 116, 166), centre_rect) # Cadre de droite
    # Dessiner le contour noir
    pygame.draw.rect(fenetre, (21, 67, 96), cadre_rect, 10)  # Le dernier argument (2) définit l'épaisseur du contour

def modifie_etat_cellule(x, y):
    """ Modifie l'état de la cellule aux coordonnées x,y (largeur, hauteur) à chaque clic de souris """
    if x <= largeur * taille_cellule and y <= hauteur * taille_cellule:
        # Calcule les indices de la cellule cliquée
        colonne = x // taille_cellule
        ligne = y // taille_cellule
        
        # Inverse l'état de la cellule
        etat[colonne][ligne] = 1 if etat[colonne][ligne] == 0 else 0
        
        # Permet d'afficher le nouvelle etat directement
        draw(), draw_button_pause()
        pygame.display.flip() 
        print(f"Cellule à ({colonne}, {ligne}) maintenant {'vivante' if etat[colonne][ligne] == 1 else 'morte'}")
    
#####################################################################################    

<a id="Save"></a>
<h2 style="color:Green">Partie Sauvegarde et Chargement </h2>

[Retour à la Table des matières](#tdm)

In [ ]:
def save_grille(fichier):
    """ Sauvegarde l'état actuel de la grille dans un fichier CSV avec pandas """
    df = pd.DataFrame(etat)
    df.to_csv(fichier, index=False, header=False)
    print(f"Grille sauvegardée dans {fichier}")

<a id="BoutonPause"></a>
<h2 style="color:Green">Bouton pause</h2>

[Retour à la Table des matières](#tdm)

In [ ]:
def draw_button_pause():
    """ Dessine le bouton pause dans le cadre de droite """
    global bouton_pause_rect
    
    # Définir la position et la taille du bouton
    bouton_pause_rect = pygame.Rect(taille_cellule * largeur + 50, 50, 300, 50)
    
    # Choisir la couleur du bouton (s’éclaircit au survol)
    couleur_bouton = (214, 234, 248)
    pygame.draw.rect(fenetre, couleur_bouton, bouton_pause_rect)  # Dessiner le bouton
    
    # Ajouter le texte "Pause" ou "Reprendre"
    font = pygame.font.SysFont(None, 30)
    texte = font.render("REPRENDRE" if en_pause else "PAUSE", True, (21, 67, 96))
    
    # Calculer la position pour centrer le texte dans le bouton
    texte_x = bouton_pause_rect.x + (bouton_pause_rect.width - texte.get_width()) // 2
    texte_y = bouton_pause_rect.y + (bouton_pause_rect.height - texte.get_height()) // 2
    
    # Dessiner le texte au centre du bouton
    fenetre.blit(texte, (texte_x, texte_y))

def toggle_pause(pos_x, pos_y):
    """ Basculer entre pause et reprise en fonction de la position du clic """
    global en_pause
    if bouton_pause_rect.collidepoint(pos_x, pos_y):  # Vérifie si le clic est sur le bouton pause
        en_pause = not en_pause  # Inverser l'état de pause
        draw_button_pause()
        pygame.display.flip()
        print("Jeu en pause" if en_pause else "Relancement du jeu")
        
        
